# Idemia's data exploration
Idemia provides keypoints on NTU videos found with mmpose. 

In [2]:
# imports
import os
import json
import torch
import numpy as np

In [3]:
# set device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device: {}".format(device))

device: cuda


In [4]:
# load data
data_dir = "data/preds/"
data_files = os.listdir(data_dir)

---
## First file discovery

In [11]:
# Open the first file in the folder to check its format
example_file = open(os.path.join(data_dir, data_files[0]))
example = json.load(example_file)

In [16]:
# keys of the output dict
example.keys()

dict_keys(['sequence_name', 'num_frames', 'frames'])

In [44]:
example['sequence_name'], example['num_frames'], type(example['frames'])

('S001C001P001R001A006_rgb', 75, list)

### First frame

In [33]:
frame = example['frames'][0]
frame_keys = frame.keys()
print(f'Keys for a frame dict : {frame_keys}')

Keys for a frame dict : dict_keys(['id', 'num_pedestrians', 'detections'])


In [39]:
id = frame['id']
num_pedestrians = frame['num_pedestrians']
print(f'Frame id and number of pedestrians {id, num_pedestrians}')

Frame id and number of pedestrians (0, 1)


In [45]:
detections = frame['detections']
type(detections), len(detections), 

(list, 1)

In [50]:
pedestrian_frame = detections[0]
print(f'Keys for a pedestrian frame : {pedestrian_frame.keys()}')

Keys for a pedestrian frame : dict_keys(['keypoints', 'keypoints_3d', 'bbox', 'pedestrian_id'])


In [60]:
pedestrian = pedestrian_frame['pedestrian_id']
bbox = pedestrian_frame['bbox']
print(f'For the pedestrian {pedestrian}, its bounding box is {bbox}.')

For the pedestrian 0, its bounding box is [965.721923828125, 282.98504638671875, 1099.129150390625, 750.9996948242188, 0.9992368221282959].


The number of value for the bounding box is quite surprising.\
According to the documentation the values corresponds to left, right, top, bottom, [score].\
Seems like (0,0) is in the bottom right corner. 
The score might be a certainty value for the bounding box (see the use of the word score in https://mmpose.readthedocs.io/en/latest/api.html).

In [73]:
keypoints = pedestrian_frame['keypoints']
print(f'There are {len(keypoints)} keypoints, each represented with {len(keypoints[0])} values.')
print('It appears the last one is the score and the other two the coordinates.')
keypoints[:3]

There are 17 keypoints, each represented with 3 values.
It appears the last one is the score and the other two the coordinates.


[[1032.425537109375, 512.4219360351562, 0.8548943400382996],
 [1009.5732421875, 509.3749694824219, 0.8316577076911926],
 [1024.80810546875, 612.971923828125, 0.8380324840545654]]

In [74]:
keypoints_3d = pedestrian_frame['keypoints_3d']
print(f'There are {len(keypoints_3d)} keypoints, each represented with {len(keypoints_3d[0])} values.')
print('This time, some values look like coordinates inside the bounding box.')
keypoints_3d[:3]


There are 17 keypoints, each represented with 3 values.
This time, some values look like coordinates inside the bounding box.


[[0.0001173038617707789, -4.1300940210931e-05, 0.5898646116256714],
 [0.09304507821798325, 0.015636632218956947, 0.5885741114616394],
 [0.032809868454933167, -0.16433845460414886, 0.2869519293308258]]

### Code study

To know more about how mmpose computes the predictions, and understand its results.

## Compare with NTU original skeleton data

In [1]:
data2D_dir = "data/nturgbd60_skeletons_2D/"
data2D_files = os.listdir(data2D_dir)

In [3]:
file0 = np.load(os.path.join(data2D_dir, data2D_files[0]))

In [13]:
print(f'Looks like the file structure is number of frames * number of joints * number of coordinates {file0.shape}')

Looks like the file structure is number of frames * number of joints * number of coordinates (103, 25, 2)


One can check that by opening the corresponding .skeleton file. And compare the coordinates to the 6th and 7th columns of a skeleton block. 

### Transform from mmpose format to original

In [32]:
incomplete_files = []
for file_name in data_files:
    with open(os.path.join(data_dir, file_name)) as f:
        skeleton_file = json.load(f)
        try:
            num_frames = skeleton_file['num_frames']
            nb_joints = len(skeleton_file['frames'][0]['detections'][0]['keypoints'])
            nb_persons = skeleton_file['frames'][0]['num_pedestrians']
            for person in range(nb_persons):
                data = np.zeros((num_frames, nb_joints, 2))
                for i_frame, frame in enumerate(skeleton_file['frames']):
                    data[i_frame] = np.array(frame['detections'][person]['keypoints'])[:, 0:2]
                np.save(os.path.join('data', 'mmpose_ntu', file_name[:-5]+f'_person{person}.npy'), data)
        except IndexError:
            incomplete_files.append(file_name)
            continue

In [33]:
len(incomplete_files)

663